In [ ]:
import os, json, time, traceback, argparse
# import openai
from tqdm.notebook import tqdm
from glob import glob
import torch
import numpy as np
import pandas as pd
import re
import transformers
from multiprocessing import Process, Pool
import multiprocessing
from collections import Counter
from fastparquet import ParquetFile

In [ ]:
# glm_device = "cuda:0"
# glm_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/chatglm3-6b", trust_remote_code=True)
# glm_model = transformers.AutoModel.from_pretrained("GPTs/chatglm3-6b", device_map="cuda:0", trust_remote_code=True).half()
# glm_model = glm_model.eval()
# # response, history = glm_model.chat(glm_tokenizer, "你好", history=[])
# def chat_glm3(glm_model, glm_tokenizer, glm_device, prompt, history=[]):
#     response, history = glm_model.chat(glm_tokenizer, prompt, history=history)
#     # print(history)
#     return response, history
# model_a_name = "chatglm3-6b"

In [ ]:
# yi_device = "cuda:0"
# yi_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/Yi-6B', use_fast=False)
# yi_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/Yi-6B', device_map=yi_device, torch_dtype='auto').eval()
# def chat_yi(yi_model, yi_tokenizer, yi_device, prompt, messages=None):
#     if messages == None:
#         messages = [{"role": "user", "content": prompt}]
#     else:
#         messages.append({"role": "user", "content": prompt})
#     input_ids = yi_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#     output_ids = yi_model.generate(input_ids.to('cuda'))
#     response = yi_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
#     messages.append({"role": "assistant", "content": response})
#     return response, messages
# model_a_name = "yi-6b"

In [ ]:
# gemma_device = "cuda:0"
# gemma_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/gemma-7b')
# gemma_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/gemma-7b', device_map=gemma_device, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2").eval()
# def chat_gemma(gemma_model, gemma_tokenizer, gemma_device, prompt, messages=None):
#     if messages == None:
#         messages = [{"role": "user", "content": prompt}]
#     else:
#         messages.append({"role": "user", "content": prompt})
#     outputs = gemma_model.generate(input_ids=gemma_tokenizer.encode(
#         gemma_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True), 
#         add_special_tokens=False, return_tensors="pt").to(gemma_model.device), max_new_tokens=150)
#     response = gemma_tokenizer.decode(outputs[0])
#     messages.append({"role": "assistant", "content": response})
#     return response, messages
# model_a_name = "gemma-7b"

In [ ]:
# phi_device = "cuda:0"
# phi_model = transformers.AutoModelForCausalLM.from_pretrained("GPTs/phi-1", torch_dtype="auto", trust_remote_code=True, device_map=phi_device)
# phi_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/phi-1", trust_remote_code=True)
# model_a_name = "phi-1b"

In [ ]:
# tiny_device = "cuda:0"
# tiny_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/TinyLlama-1.1B', device_map=tiny_device, torch_dtype='auto').eval()
# tiny_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/TinyLlama-1.1B', truncation_side='left')
# model_b_name = 'TinyLlama-1.1B'

In [ ]:
# baichuan_device = "cuda:0"
# baichuan_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser("~/GPTs/Baichuan2-7B"), trust_remote_code=True)
# baichuan_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser("~/GPTs/Baichuan2-7B"), device_map=baichuan_device, torch_dtype=torch.bfloat16, trust_remote_code=True).half()
# baichuan_model.generation_config = transformers.GenerationConfig.from_pretrained(os.path.expanduser("~/GPTs/Baichuan2-7B"))
# baichuan_model = baichuan_model.eval()
# baichuan_name = "Baichuan2-7B"

In [ ]:
# qw_device = "cuda:0"
# qw_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser("~/GPTs/qwen1.5-7b"), torch_dtype="auto", device_map=qw_device, attn_implementation="flash_attention_2").eval()
# qw_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser("~/GPTs/qwen1.5-7b"))
# # Specify hyperparameters for generation. But if you use transformers>=4.32.0, there is no need to do this.
# # model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-14B-Chat", trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
# """
# def chat_qwen(qw_model, qw_tokenizer, qw_device, prompt, messages=None):
#     if messages is None:
#         messages = [{"role": "system", "content": "You are a helpful assistant."}]
#     messages.append({"role": "user", "content": prompt})
#     new_input = qw_tokenizer([qw_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)], return_tensors="pt").to(qw_device)
#     resp = qw_tokenizer.batch_decode([output_ids[len(input_ids):] for input_ids, output_ids in zip(new_input.input_ids, qw_model.generate(new_input.input_ids, max_new_tokens=512, pad_token_id=qw_tokenizer.eos_token_id))], 
#                                  skip_special_tokens=True)[0]
#     messages.append({"role": "assistant", "content": resp})
#     return resp, messages
# """
# qw_name = "qwen1.5-7b"

In [ ]:
qw_device = "cuda:0"
qw_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser("~/GPTs/Qwen2-7B-Instruct"), torch_dtype="auto", device_map=qw_device, attn_implementation="flash_attention_2")
qw_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser("~/GPTs/Qwen2-7B-Instruct"))
# Specify hyperparameters for generation. But if you use transformers>=4.32.0, there is no need to do this.
# qw_model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-14B-Chat", trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
qw_name = "qwen2-7b"

In [ ]:
llama_device = "cuda:1"
llama_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser('~/GPTs/llama3.1-8B-Instruct'), device_map=llama_device, torch_dtype='auto', attn_implementation="flash_attention_2").eval()
llama_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser('~/GPTs/llama3.1-8B-Instruct'))#, truncation_side='left')
llama_name = 'llama3.1-8b'

In [ ]:
# mistral_device = "cpu"
mistral_device = "cuda:0"
mistral_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser("~/GPTs/Mistral-7B-Instruct-v0.3"), torch_dtype="auto", device_map=mistral_device,
                                                                 attn_implementation="flash_attention_2"
                                                                 )
mistral_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser("~/GPTs/Mistral-7B-Instruct-v0.3"))
mistral_name = "Mistral-7B-v0.3"

In [ ]:
# mistral_device = "cpu"
# mistral_model = transformers.AutoModelForCausalLM.from_pretrained(os.path.expanduser("~/GPTs/Mistral-v0.2-7B"), torch_dtype="auto", device_map=mistral_device, attn_implementation="flash_attention_2").eval()
# mistral_tokenizer = transformers.AutoTokenizer.from_pretrained(os.path.expanduser("~/GPTs/Mistral-v0.2-7B"))
# mistral_name = "Mistral-v0.2-7B"

## Brainstorming

In [ ]:
# utils
def get_result_df(dataset_name, models_info, timestamp, max_round, save_last_n_round, result_dir):
    # model_num = len(models_info)
    model_name_list = list(map(lambda x: x['name'], models_info))
    result_csv_filename = f"{dataset_name}_{'_'.join(model_name_list)}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=(['ith','label']+[f"{n}_first_ans" for n in model_name_list]+['game_ans','rounds']))
        result_df.to_csv(csv_path, index=False)
    return result_df

def to_result_df(dataset_name, models_info, 
                 timestamp, max_round, save_last_n_round, result_dir, # runtime
                 data_row, # to result_csv
                 ):
    model_name_list = list(map(lambda x: x['name'], models_info))
    result_csv_filename = f"{dataset_name}_{'_'.join(model_name_list)}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=(['ith','label']+[f"{n}_first_ans" for n in model_name_list]+['game_ans','rounds']))
        result_df.to_csv(csv_path, index=False)
    result_df.loc[len(result_df)] = data_row
    # result_df = pd.concat([result_df, pd.DataFrame([{
    #         'ith':ith, 'label':label, 'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
    #         'game_ans':game_ans,'rounds':rounds}])],ignore_index=True)
    result_df.to_csv(csv_path, index=False)
    return result_df

def save_dialogs(dialog_items, converged, split_mark, dataset_name, idx, models_info, timestamp, result_dir):
    dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
    model_name_list = list(map(lambda x: x['name'], models_info))
    if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
    dialog_filename = f"{dataset_name}_{'_'.join(model_name_list)}_{timestamp}_{idx}_{'Y' if converged else 'N'}.txt"
    dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
    for resp in dialog_items:
        dialog_fp.write(resp+'\n')
        dialog_fp.write(split_mark)
    dialog_fp.close()

In [ ]:
def brainstorm(models_info, max_round, save_last_n_round, timestamp, result_dir, # runtime
               dataset_df, dataset_name, equal_func, init_prompt_template, game_prompt_template, parse_ans, # data & prompt & utils
               # parallel_questions: int, # run *2 chat process
               # start_offset=0 # another 2 GPUs node!
              ):
    dialog_item_split = "**/-|+|=\**\n"
    model_num = len(models_info)
    
    # pool = Pool(processes=parallel_questions*2, maxtasksperchild=1)
    # proc_pointer = [[None, None] for _ in range(parallel_questions)]
    # runtime_record = [reset_runtime() for _ in range(parallel_questions)]
    result_df = get_result_df(dataset_name, models_info, timestamp, max_round, save_last_n_round, result_dir)
    # pbar = tqdm(range(len(dataset_df))) #total=len(dataset_df)-start_offset)
    restart = True
    # question_index = start_offset
    # finished_count = 0
    for idx in (pbar:=tqdm(range(len(dataset_df)))):
        if idx in result_df['ith'].values:
            # pbar.update(1)
            continue
        if len(result_df)>0:
            if restart:
                label_so_far = result_df.label.to_list()
                model_1st_column_names = result_df.columns[2:-2]
                models_so_far = [getattr(result_df, col_name).to_list() for col_name in model_1st_column_names]
                game_so_far = result_df.game_ans.to_list()
                model_1st_corrects = [list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, so_far))).count(True) for so_far in models_so_far]
                # a_correct = list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, model_a_so_far))).count(True)
                # b_correct = list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, model_b_so_far))).count(True)
                game_correct = list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, game_so_far))).count(True)
                # consensus_wrong = list(map(lambda x: (not equal_func(x[0],x[1])) and (x[1] != NOT_CONVERGE_MARK), 
                #                            zip(label_so_far, game_so_far))).count(True)
                restart = False
        else:
            model_1st_corrects = [0] * model_num
            game_correct = 0
        # idx-th problem, game rounds
        answers_for_majority = []
        # refractor dataframe before this calling
        # choice: <Q-stem><choices> | <choice>
        # gsm like: <Question> | <numerical number>
        question = dataset_df.iloc[idx].question
        subject = dataset_df.iloc[idx].subject
        answer = dataset_df.iloc[idx].answer
        question = init_prompt_template(dataset_df.iloc[idx].question, subject)
        dialog_item = [question]
        for round in range(max_round):
            torch.cuda.empty_cache()
            pool = Pool(processes=model_num, maxtasksperchild=1)
            if round == 0:
                results = [pool.apply_async(d['chatcaller'], (d['obj'], d['tokenizer'], d['device'], question,) ) for d in models_info]#[:-1]]
                # # 3rd model in main thread
                # d = models_info[-1]
                # results_last = d['chatcaller'](d['obj'], d['tokenizer'], d['device'], question,)
            else:
                if round > save_last_n_round:    # shave history
                    histories = [(history[:(3 if history[0]['role'] == "system" else 2)] + history[-2*save_last_n_round:])
                                 for _, history in resp_history]
                else:
                    histories = [history for _, history in resp_history]
                # get responses of all models from last round
                resps = []
                for i, (resp, _) in enumerate(resp_history):
                    resps.append(resp)
                # make game prompts
                game_prompts = []
                for i in range(model_num):
                    resps_game = resps[:].pop(i)
                    game_prompts.append(game_prompt_template(resps_game))
                results = [pool.apply_async(d['chatcaller'], (d['obj'], d['tokenizer'], d['device'], game_prompts[i], histories[i]) ) for i,d in enumerate(models_info)]
                # pool.close()
                # pool.join()
                # resp_history = [pointer.get() for pointer in results]
                # # 3rd model in main thread
                # d = models_info[-1]
                # results_last = d['chatcaller'](d['obj'], d['tokenizer'], d['device'], game_prompts[-1], histories[-1])
            pool.close()
            pool.join()
            resp_history = [pointer.get() for pointer in results]
            # # 3rd model in main thread
            # resp_history.append(results_last)
            parsed_answers = []
            # save dialog
            for i,(r,_) in enumerate(resp_history):
                dialog_item.append(f"{models_info[i]['name']}: {r}")
                p_r = parse_ans(r)
                parsed_answers.append(p_r)
                answers_for_majority.append(p_r)
            # for first round
            if round == 0: 
                first_answers = parsed_answers[:]
                for i,a in enumerate(parsed_answers[:]):
                    if equal_func(a, answer):
                       model_1st_corrects[i] += 1
            # set `converged`
            converged = True
            for i in range(len(parsed_answers)-1):
                converged = converged & (equal_func(parsed_answers[i], parsed_answers[i+1]))
                if not converged: break
            # finish
            if converged or round == max_round -1:
                if converged:
                    if equal_func(parsed_answers[0], answer): game_correct += 1
                    # else: consensus_wrong += 1
                    game_ans = parsed_answers[0]
                else: # not converged, reach max_round. majority vote when no converge
                    try:
                        v = Counter(answers_for_majority)
                        game_ans = v.most_common(1)[0][0]
                    except:
                        choices = ["A","B","C","D"]
                        if "or (E)" in question: choices.append("E")
                        game_ans = np.random.choice(choices)
                    if equal_func(game_ans, answer): game_correct += 1
                result_df = to_result_df(dataset_name, models_info, timestamp, max_round, save_last_n_round, result_dir,
                                         data_row=[idx, answer]+first_answers+[game_ans, round])
                save_dialogs(dialog_item, converged, dialog_item_split, dataset_name,  idx, models_info, timestamp, result_dir)
                del resp_history
                break
        each_model_str = ", ".join([f"{d['name']}={model_1st_corrects[i]/(idx+1):.3f}" for i,d in enumerate(models_info)])
        pbar.set_description(f'{dataset_name} Game={game_correct/(idx+1):.3f}, {each_model_str}')
        # {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
    # print(f"Subject: {subject} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
    label_so_far = result_df.label.to_list()
    model_1st_column_names = result_df.columns[2:-2]
    models_so_far = [getattr(result_df, col_name).to_list() for col_name in model_1st_column_names]
    game_so_far = result_df.game_ans.to_list()
    model_1st_corrects = [list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, so_far))).count(True) for so_far in models_so_far]
    game_correct = list(map(lambda x: equal_func(x[0],x[1]), zip(label_so_far, game_so_far))).count(True)
    each_model_str = ", ".join([f"{d['name']}={model_1st_corrects[i]/(len(result_df)):.3f}" for i,d in enumerate(models_info)])
    print(f'{dataset_name} Game={game_correct/(len(result_df)):.3f}, {each_model_str}')

### MMLU

In [ ]:
# MMLU 
def mmlu_init_prompt(q, subject_name):
    return f"You are an expert in {subject_name}. Can you answer the following question as accurately as possible? {q} Explain your answer, putting the answer choice in the form (A) or (B) or (C) or (D) at the end of your response."

def game_prompt_mmlu(another_resps):
    # if len(agents) == 0:
    #     return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}
    p = "These are the solutions to the problem from other agents: "
    for r in another_resps:
        p += "\n\nOne agent solution: ```{}```".format(r)
    p += """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer choice in the form (A) or (B) or (C) or (D) at the end of your response."""
    return p

def parse_choice_ans(input_str):
    pattern = r'\(\s{0,}(\w)\s{0,}\)'
    matches = re.findall(pattern, input_str)
    solution = None
    for match_str in matches[::-1]:
        solution = match_str.upper()
        if solution: break
    return solution

def mmlu_df():
    task_paths = glob("Corpus/MMLU/test/*.csv")
    subjects = list(map(lambda x: x.split('/')[-1].split('_test.csv')[0], task_paths))
    mmlu_data_list = []
    for task_path, subject in zip(task_paths, subjects):
        df = pd.read_csv(task_path, header=None)
        for ix in range(len(df)):
            question = df.iloc[ix, 0]
            a = df.iloc[ix, 1]
            b = df.iloc[ix, 2]
            c = df.iloc[ix, 3]
            d = df.iloc[ix, 4]
            # subject, question, answer
            mmlu_data_list.append([subject.replace("_", " "), f"{question}: (A) {a}, (B) {b}, (C) {c}, (D) {d}", df.iloc[ix, 5]])
    mmlu_df = pd.DataFrame(mmlu_data_list, columns=['subject', 'question', 'answer'])
    return mmlu_df

def mmlu_equal(a, b): 
    if isinstance(a, str) and isinstance(b, str):
        return a.upper() == b.upper()
    else: return a == b

In [ ]:
from ChatLLM import chat_baichuan, chat_qwen2, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    # model
    models_info = [{"name": qw_name, "obj": qw_model, "device": qw_device, "tokenizer": qw_tokenizer, "chatcaller": chat_qwen2},
                  {"name": llama_name, "obj": llama_model, "device": llama_device, "tokenizer": llama_tokenizer, "chatcaller": chat_llama},
                  # {"name": baichuan_name, "obj": baichuan_model, "device": baichuan_device, "tokenizer": baichuan_tokenizer, "chatcaller": chat_baichuan},
                  {"name": mistral_name, "obj": mistral_model, "device": mistral_device, "tokenizer": mistral_tokenizer, "chatcaller": chat_mistral}]
    # hyper params
    max_round = 5
    timestamp = 20240808
    result_dir = "Results/GameLLM"
    save_last_n_round = 1
    if not os.path.exists(result_dir): os.mkdir(result_dir)
    # data
    dataset_df = mmlu_df()
    dataset_name = "MMLU"
    print("MMLU data merged.")
    # main
    while True:
        try:
            brainstorm(
                models_info, max_round, save_last_n_round, timestamp, result_dir,# runtime
                dataset_df=dataset_df, dataset_name=dataset_name, # data
                equal_func=mmlu_equal, init_prompt_template=mmlu_init_prompt, game_prompt_template=game_prompt_mmlu, 
                parse_ans=parse_choice_ans, # prompt & utils
            )
            break
        except torch.cuda.OutOfMemoryError:
            print('OOM')
        # except ValueError:
        #     pass

### GSM

In [ ]:
# MMLU 
def gsm_init_prompt(q, subject_name):
    question = f"You are an expert in math. Please solve the following math problem step by step. {q}\nExplain your reasoning. "
    question += "Your final answer should be a single numerical number in the form of {answer} at the end of your response."
    question += "Make sure that the last numerical number in your response is your final answer."
    return question

def game_prompt_gsm(another_resps):
    # if len(agents) == 0:
    #     return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}
    p = "These are the solutions to the problem from other agents: "
    for r in another_resps:
        p += "\n\nOne agent solution: ```{}```".format(r)
    p += "\n\n Using the solutions from other agents as additional information, can you provide your answer to the math problem? "
    p += "Examine your solution and other agents' solution step by step. "
    p += "Your final answer should be a single numerical number in the form of {answer} at the end of your response. "
    p += "Make sure that the last numerical number in your response is your final answer."
    return p

def parse_gsm_ans(s):
    s = s.replace(",", "").replace(r"\$", "")    # case: 70,000 $18
    pattern1 = r"\{[+-]?\d+\.?\d*\}"    # fix minus number
    pattern2 = r"[+-]?\d+\.?\d*"
    if (matches := re.findall(pattern1, s)):
        return matches[-1].replace("{", "").replace("}", "")
    elif(matches := re.findall(pattern2, s)):
        return matches[-1].replace("{", "").replace("}", "")
    return None

import nemo_skills
from nemo_skills.code_execution.math_grader import * #math_equal
nemo_symbolic_equal = nemo_skills.code_execution.math_grader.symbolic_equal
def my_symbolic_equal(_a, _b, tolerance, timeout=10.0):
    import sympy
    from sympy.parsing.latex import parse_latex
    from sympy.parsing.sympy_parser import parse_expr
    # expr compare style
    try:
        a = parse_expr(_a)
        b = parse_expr(_b)
        try:
            with time_limit(timeout*2):
                if sympy.simplify(a - b) == 0: return True
        except Exception:
            pass
        try:
            with time_limit(timeout):
                if isclose(sympy.N(a), sympy.N(b), rel_tol=tolerance): return True
        except Exception:
            pass
    except Exception:
        pass
    # LaTeX compare style
    try:
        a = parse_latex(_a)
        b = parse_latex(_b)
        return a.equals(b)
    except Exception:    # I do not understand
        pass
    return False

nemo_skills.code_execution.math_grader.symbolic_equal = my_symbolic_equal
# def math_ans_is_equal(resp, label): 
#     if str(resp).strip() == str(label).strip(): return True
#     return math_equal(str(resp), str(label))

def gsm_ans_is_equal(resp, label):
    # if resp is not None and label is not None:
    #     return np.abs(float(label)-float(resp))<1e-5
    # return False
    if str(resp).strip() == str(label).strip(): return True
    return math_equal(str(resp), str(label))

def gsm_df():
    df = ParquetFile(os.path.expanduser('~/Corpus/gsm8k/main/test-00000-of-00001.parquet')).to_pandas()
    gsm_data_list = []
    for ix in range(len(df)):
            question = df.iloc[ix, 0]
            answer = df.iloc[ix, 1].split("####")[-1].strip()
            # subject, question, answer
            gsm_data_list.append(["math", question, answer])
    gsm_df = pd.DataFrame(gsm_data_list, columns=['subject', 'question', 'answer'])
    return gsm_df

#####################################################

from ChatLLM import chat_baichuan, chat_qwen2, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    # model
    models_info = [{"name": qw_name, "obj": qw_model, "device": qw_device, "tokenizer": qw_tokenizer, "chatcaller": chat_qwen2},
                  {"name": llama_name, "obj": llama_model, "device": llama_device, "tokenizer": llama_tokenizer, "chatcaller": chat_llama},
                  # {"name": baichuan_name, "obj": baichuan_model, "device": baichuan_device, "tokenizer": baichuan_tokenizer, "chatcaller": chat_baichuan},
                  {"name": mistral_name, "obj": mistral_model, "device": mistral_device, "tokenizer": mistral_tokenizer, "chatcaller": chat_mistral}]
    # hyper params
    max_round = 8
    timestamp = 20240809
    result_dir = "Results/GameLLM"
    save_last_n_round = 1
    if not os.path.exists(result_dir): os.mkdir(result_dir)
    # data
    dataset_df = gsm_df()
    dataset_name = "GSM"
    print("GSM data ready.")
    # main
    while True:
        try:
            brainstorm(
                models_info, max_round, save_last_n_round, timestamp, result_dir,# runtime
                dataset_df=dataset_df, dataset_name=dataset_name, # data
                equal_func=gsm_ans_is_equal, init_prompt_template=gsm_init_prompt, game_prompt_template=game_prompt_gsm, 
                parse_ans=parse_gsm_ans, # prompt & utils
            )
            break
        except torch.cuda.OutOfMemoryError:
            print('OOM')
        # except ValueError:
        #     pass

### Arc-e/c

In [ ]:
# MMLU 
def arc_init_prompt(question, subject):
    return question

def _arc_init_prompt(question, answer, choice_text, choice_label):    
    if answer.isdigit():
        answer = chr(ord("A")-1+int(answer)) # map digit to A B C D E
    question = f"Can you answer the following question as accurately as possible? {question}: "
    for i,choice in enumerate(choice_text):
        if i>0: question += " "
        question += f"({chr(ord('A')+i)}) {choice}"
    question += " Explain your answer, putting the best one answer choice in the form "
    answer_form = ""
    for i in range(len(choice_label)):
        if i>0: answer_form += "or "
        answer_form += f"({chr(ord('A')+i)}) "
    question += answer_form + "at the end of your response."
    # print(question, answer)
    # raise RuntimeError
    return question, answer

def game_prompt_arc(another_resps):
    p = "These are the solutions to the problem from other agents: "
    for r in another_resps:
        p += "\n\nOne agent solution: ```{}```".format(r)
    p += "\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. "
    p += f"Put your answer choice in the form (A) or (B) or (C) or (D) or (E) at the end of your response."
    p += f"Make sure that the last term in the form (A) or (B) or (C) or (D) or (E) in your response is your final answer."
    return p

def arc_ans_is_equal(resp, label): return resp == label

def arc_df(difficulty):
    df = ParquetFile(os.path.expanduser(f'~/Corpus/ai2_arc/ARC-{difficulty}/test-00000-of-00001.parquet')).to_pandas()
    arc_data_list = []
    for ix in range(len(df)):
        question = df.iloc[ix, 1]
        answer = df.iloc[ix, 2]
        question, answer = _arc_init_prompt(question, answer, df.iloc[ix, 3], df.iloc[ix, 4])
        arc_data_list.append([f"ARC-{difficulty}", question, answer])
    arc_df = pd.DataFrame(arc_data_list, columns=['subject', 'question', 'answer'])
    # print(arc_df.iloc[0,1])
    return arc_df

#####################################################

from ChatLLM import chat_baichuan, chat_qwen2, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    # model
    models_info = [{"name": qw_name, "obj": qw_model, "device": qw_device, "tokenizer": qw_tokenizer, "chatcaller": chat_qwen2},
                  {"name": llama_name, "obj": llama_model, "device": llama_device, "tokenizer": llama_tokenizer, "chatcaller": chat_llama},
                  # {"name": baichuan_name, "obj": baichuan_model, "device": baichuan_device, "tokenizer": baichuan_tokenizer, "chatcaller": chat_baichuan},
                  {"name": mistral_name, "obj": mistral_model, "device": mistral_device, "tokenizer": mistral_tokenizer, "chatcaller": chat_mistral}]
    # hyper params
    max_round = 8
    timestamp = 20240810
    result_dir = "Results/GameLLM"
    save_last_n_round = 1
    if not os.path.exists(result_dir): os.mkdir(result_dir)
    # data
    # dataset_df = arc_df("Easy")
    # dataset_name = "ARC-e"
    # print(f"{dataset_name} data ready.")
    # # main
    # while True:
    #     try:
    #         brainstorm(
    #             models_info, max_round, save_last_n_round, timestamp, result_dir,# runtime
    #             dataset_df=dataset_df, dataset_name=dataset_name, # data
    #             equal_func=arc_ans_is_equal, init_prompt_template=arc_init_prompt, game_prompt_template=game_prompt_arc, 
    #             parse_ans=parse_choice_ans, # prompt & utils
    #         )
    #         break
    #     except torch.cuda.OutOfMemoryError:
    #         print('OOM')
    # ############################################
    dataset_df = arc_df("Challenge")
    dataset_name = "ARC-c"
    print(f"{dataset_name} data ready.")
    # main
    while True:
        try:
            brainstorm(
                models_info, max_round, save_last_n_round, timestamp, result_dir,# runtime
                dataset_df=dataset_df, dataset_name=dataset_name, # data
                equal_func=arc_ans_is_equal, init_prompt_template=arc_init_prompt, game_prompt_template=game_prompt_arc, 
                parse_ans=parse_choice_ans, # prompt & utils
            )
            break
        except torch.cuda.OutOfMemoryError:
            print('OOM')
    

In [ ]:
difficulty = 'Easy'
df = ParquetFile(os.path.expanduser(f'~/Corpus/ai2_arc/ARC-{difficulty}/test-00000-of-00001.parquet')).to_pandas()
df.iloc[0]

## MMLU

In [ ]:
#"""in the form (X) <-> in between the brakets """
"""put your answer ++choice++
A) -> (A)
in the form (A) or (B) or (C) or (D)
"""
def parse_mmlu(df, ix, subject_name):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]
    question = "You are an expert in {}. Can you answer the following question as accurately as possible? {}: (A) {}, (B) {}, (C) {}, (D) {} Explain your answer, putting the answer choice in the form (A) or (B) or (C) or (D) at the end of your response.".format(subject_name, question, a, b, c, d)
    answer = df.iloc[ix, 5]
    return question, answer

def game_prompt_mmlu(another_resp):
    # if len(agents) == 0:
    #     return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}
    prefix_string = "These are the solutions to the problem from other agents: \n\n One agent solution: ```{}```".format(another_resp)
    prefix_string += """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer choice in the form (A) or (B) or (C) or (D) at the end of your response."""
    return prefix_string

def parse_choice_ans(input_str):
    pattern = r'\(\s{0,}(\w)\s{0,}\)'
    matches = re.findall(pattern, input_str)
    solution = None
    for match_str in matches[::-1]:
        solution = match_str.upper()
        if solution: break
    return solution

In [ ]:
def main_run(model_a, model_b, tkz_a, tkz_b, device_a, device_b, chat_model_a, chat_model_b):
    # task paths
    task_paths = glob("Corpus/MMLU/test/*.csv")
    subjects = list(map(lambda x: x.split('/')[-1].split('_test.csv')[0], task_paths))
    # build result df
    result_csv_filename = f"MMLU_{model_a_name}_{model_b_name}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=['subject','ith','label','a_first_ans','b_first_ans','a_game_ans','b_game_ans','rounds'])
        result_df.to_csv(csv_path, index=False)
    def write_to_result_df(result_df, subject, i, answer, a_first_ans, b_first_ans, a_game_answer, b_game_answer, g):
        result_df = pd.concat([result_df, pd.DataFrame([{
                'subject': subject, 'ith':i, 'label':answer,
                'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
                'a_game_ans':a_game_answer,'b_game_ans':b_game_answer,'rounds':g
            }])],ignore_index=True)
        result_df.to_csv(os.path.join(result_dir, result_csv_filename), index=False)
        return result_df
    
    for task_path, subject in zip(task_paths, subjects):
        # fetch statistic
        df = pd.read_csv(task_path, header=None)
        if (result_lines:=len(result_df[result_df['subject']==subject]))>0:
            if result_lines==len(df): continue
            label_so_far = result_df[result_df['subject']==subject].label.to_list()
            model_a_so_far = result_df[result_df['subject']==subject].a_first_ans.to_list()
            model_b_so_far = result_df[result_df['subject']==subject].b_first_ans.to_list()
            game_a_so_far = result_df[result_df['subject']==subject].a_game_ans.to_list()
            game_b_so_far = result_df[result_df['subject']==subject].b_game_ans.to_list()
            a_correct = list(map(lambda x: x[0]==x[1], zip(label_so_far, model_a_so_far))).count(True)
            b_correct = list(map(lambda x: x[0]==x[1], zip(label_so_far, model_b_so_far))).count(True)
            game_correct = list(map(lambda x: x[0]==x[1]==x[2], zip(label_so_far, game_a_so_far, game_b_so_far))).count(True)
            # subject_start_idx = len(label_so_far)
            # if len(df) == subject_start_idx:
            #     continue
        else:
            a_correct, b_correct, game_correct, subject_start_idx = 0,0,0,0
        # print(f"Current subjuect: {subject}")
        # into one subject
        subj_slice = result_df[result_df['subject']==subject]
        for idx in (pbar:=tqdm(range(len(df)))):
            if len(subj_slice)>0:    # skip processed line by line
                if idx in subj_slice['ith'].values: continue
            # save dialog
            dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
            if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
            dialog_filename = f"{model_a_name}_{model_b_name}_{timestamp}_{subject}_{idx}_LastOnly.txt"
            dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
            # game rounds
            answers_for_majority = []
            question, answer = parse_mmlu(df, idx, subject.replace("_", " "))
            for round in range(max_round):
                if round == 0:
                    pool = Pool(processes=2)
                    result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, question,))
                    result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, question,))
                    pool.close()
                    pool.join()
                    a_resp, a_history = result_a.get()
                    b_resp, b_msgs = result_b.get()
                    # a_resp, a_history = chat_model_a(model_a, tkz_a, device_a, question)
                    # b_resp, b_msgs = chat_model_b(model_b, tkz_b, device_b, question)
                else:    # ans_a != ans_b
                    if round > save_last_n_round:    # shave history
                        a_history = a_history[:(3 if a_history[0]['role'] == "system" else 2)] + a_history[-2*save_last_n_round:]
                        b_msgs = b_msgs[:(3 if b_msgs[0]['role'] == "system" else 2)] + b_msgs[-2*save_last_n_round:]
                    game_a_prompt, game_b_prompt = game_prompt_mmlu(b_resp), game_prompt_mmlu(a_resp)
                    # a_resp, a_history = chat_model_a(model_a, tkz_a, device_a, game_a_prompt, a_history)
                    # b_resp, b_msgs = chat_model_b(model_b, tkz_b, device_b, game_b_prompt, b_msgs)
                    pool = Pool(processes=2)
                    result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, game_a_prompt, a_history,))
                    result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, game_b_prompt, b_msgs,))
                    pool.close()
                    pool.join()
                    a_resp, a_history = result_a.get()
                    b_resp, b_msgs = result_b.get()
                # save response
                dialog_fp.write(f"a: {a_resp}\nb: {b_resp}\n\n")
                ans_a, ans_b = parse_choice_ans(a_resp), parse_choice_ans(b_resp)
                # save answer for voting
                if 'MMLU' in task_path.upper():
                    if ans_a in ['A', 'B', 'C', 'D']: answers_for_majority.append(ans_a)
                    if ans_b in ['A', 'B', 'C', 'D']: answers_for_majority.append(ans_b)
                # control game flow
                if round == 0: 
                    a_first_ans, b_first_ans = ans_a, ans_b
                    if ans_a == answer: a_correct += 1
                    if ans_b == answer: b_correct += 1
                if ans_a == ans_b:
                    if ans_a == answer:
                        game_correct += 1
                    result_df = write_to_result_df(result_df, subject, idx, answer, a_first_ans, b_first_ans, ans_a, ans_b, round)
                    break
                # majority vote when no converge
                if round == max_round -1:
                    try:
                        v = Counter(answers_for_majority)
                        vote_result = v.most_common(1)[0][0]
                    except:
                        vote_result = np.random.choice(["A","B","C","D"])
                    result_df = write_to_result_df(result_df, subject, idx, answer, a_first_ans, b_first_ans, vote_result, vote_result, round)
                    answers_for_majority.clear()
                    break
            # Destructor
            del a_history, b_msgs
            dialog_fp.close()
            pbar.set_description(f"Subject: {subject} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
        print(f"Subject: {subject} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")

In [ ]:
from ChatLLM import chat_baichuan, chat_qwen, chat_phi, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    while True:
        try:
            main_run(
                model_a=baichuan_model, 
                model_b=mistral_model, 
                tkz_a=baichuan_tokenizer, 
                tkz_b=mistral_tokenizer, 
                device_a=baichuan_device, 
                device_b=mistral_device, 
                chat_model_a=chat_baichuan, 
                chat_model_b=chat_mistral
            )
            break
        except torch.cuda.OutOfMemoryError:
            pass
        except ValueError:
            pass

## GSM8k

In [ ]:
save_last_n_round = 1

from fastparquet import ParquetFile
def parse_gsm(df, ix):
    question = df.iloc[ix, 0]
    answer = df.iloc[ix, 1].split("####")[-1].strip()
    question = f"You are an expert in math. Please solve the following math problem step by step. {question}\nExplain your reasoning. "
    question += "Your final answer should be a single numerical number in the form of {answer} at the end of your response."
    question += "Make sure that the last numerical number in your response is your final answer."
    return question, answer

def game_prompt_gsm(another_resp):
    prefix_string = "These are the solutions to the problem from other agents: \n\n One agent solution: ```{}```".format(another_resp)
    prefix_string += "\n\n Using the solutions from other agents as additional information, can you provide your answer to the math problem? "
    prefix_string += "Examine your solution and other agents' solution step by step. "
    prefix_string += "Your final answer should be a single numerical number in the form of {answer} at the end of your response. "
    prefix_string += "Make sure that the last numerical number in your response is your final answer."
    return prefix_string

def parse_gsm_ans(s):
    s = s.replace(",", "").replace(r"\$", "")    # case: 70,000 $18
    pattern1 = r"\{[+-]?\d+\.?\d*\}"    # fix minus number
    pattern2 = r"[+-]?\d+\.?\d*"
    if (matches := re.findall(pattern1, s)):
        return matches[-1].replace("{", "").replace("}", "")
    elif(matches := re.findall(pattern2, s)):
        return matches[-1].replace("{", "").replace("}", "")
    return None

def gsm_ans_is_equal(resp, label):
    if resp is not None and label is not None:
        return np.abs(float(label)-float(resp))<1e-5
    return False

# for ix in range(len(gsm_test_df)):
#     q, a = parse_gsm(gsm_test_df,ix)
#     if a[0] == '-': print(a)
#     # if "####" in gsm_test_df.iloc[ix, 1]: print(ix)

def main_gsm(model_a, model_b, tkz_a, tkz_b, device_a, device_b, chat_model_a, chat_model_b):
    gsm_test_df = ParquetFile(os.path.expanduser('~/Corpus/gsm8k/main/test-00000-of-00001.parquet')).to_pandas()
    # build result df
    result_csv_filename = f"GSM_{model_a_name}_{model_b_name}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=['ith','label','a_first_ans','b_first_ans','a_game_ans','b_game_ans','rounds'])
        result_df.to_csv(csv_path, index=False)
    def gsm_to_result_df(result_df, i, answer, a_first_ans, b_first_ans, a_game_answer, b_game_answer, g):
        result_df = pd.concat([result_df, pd.DataFrame([{
                'ith':i, 'label':answer,
                'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
                'a_game_ans':a_game_answer,'b_game_ans':b_game_answer,'rounds':g
            }])],ignore_index=True)
        result_df.to_csv(os.path.join(result_dir, result_csv_filename), index=False)
        return result_df

    restart = True
    for idx in (pbar:=tqdm(range(len(gsm_test_df)))):
        if len(result_df)>0:    # skip processed line by line
            if idx in result_df['ith'].values: continue
            if restart:
                label_so_far = result_df.label.to_list()
                model_a_so_far = result_df.a_first_ans.to_list()
                model_b_so_far = result_df.b_first_ans.to_list()
                game_a_so_far = result_df.a_game_ans.to_list()
                game_b_so_far = result_df.b_game_ans.to_list()
                a_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]), zip(label_so_far, model_a_so_far))).count(True)
                b_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]), zip(label_so_far, model_b_so_far))).count(True)
                game_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]) and gsm_ans_is_equal(x[2],x[1]), 
                                        zip(label_so_far, game_a_so_far, game_b_so_far))).count(True)
                restart = False
        else:
            a_correct, b_correct, game_correct, subject_start_idx = 0,0,0,0
        # save dialog
        dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
        if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
        dialog_filename = f"GSM_{model_a_name}_{model_b_name}_{timestamp}_{idx}_LastOnly.txt"
        dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
        # game rounds
        answers_for_majority = []
        question, answer = parse_gsm(gsm_test_df,idx)
        for round in range(max_round):
            if round == 0:
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, question,))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, question,))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            else:    # ans_a != ans_b
                if round > save_last_n_round:    # shave history
                    # print(b_msgs)
                    # return
                    a_history = a_history[:(3 if a_history[0]['role'] == "system" else 2)] + a_history[-2*save_last_n_round:]
                    b_msgs = b_msgs[:(3 if b_msgs[0]['role'] == "system" else 2)] + b_msgs[-2*save_last_n_round:]
                game_a_prompt, game_b_prompt = game_prompt_gsm(b_resp), game_prompt_gsm(a_resp)
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, game_a_prompt, a_history))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, game_b_prompt, b_msgs))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            # save response
            dialog_fp.write(f"a: {a_resp}\nb: {b_resp}\n\n")
            ans_a, ans_b = parse_gsm_ans(a_resp), parse_gsm_ans(b_resp)
            # save answer for voting
            answers_for_majority.append(ans_a)
            answers_for_majority.append(ans_b)
            # control game flow
            if round == 0: 
                a_first_ans, b_first_ans = ans_a, ans_b
                if gsm_ans_is_equal(ans_a, answer.replace(',',"")): a_correct += 1
                if gsm_ans_is_equal(ans_b, answer.replace(',',"")): b_correct += 1
            if gsm_ans_is_equal(ans_a, ans_b):
                if gsm_ans_is_equal(ans_a, answer.replace(',',"")): game_correct += 1
                result_df = gsm_to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, ans_a, ans_b, round)
                break
            # majority vote when no converge
            if round == max_round -1:
                try:
                    v = Counter(answers_for_majority)
                    vote_result = v.most_common(1)[0][0]
                except:
                    vote_result = None
                result_df = gsm_to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, vote_result, vote_result, round)
                answers_for_majority.clear()
                # Destructor
                del a_history, b_msgs
                dialog_fp.close()
                break
        pbar.set_description(f"GSM Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
    print(f"GSM Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")

In [ ]:
from ChatLLM import chat_baichuan, chat_qwen, chat_phi, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    while True:
        try:
            main_gsm(
                model_a=baichuan_model, 
                model_b=mistral_model, 
                tkz_a=baichuan_tokenizer, 
                tkz_b=mistral_tokenizer, 
                device_a=baichuan_device, 
                device_b=mistral_device, 
                chat_model_a=chat_baichuan, 
                chat_model_b=chat_mistral
            )
            break
        except torch.cuda.OutOfMemoryError:
            pass
        # except ValueError:
        #     pass